In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets seqeval
!python -m pip install -q 'git+https://github.com/facebookresearch/detectron2.git'


  Installing build dependencies ... done
  Getting requirements to build wheel ... canceledERROR: Operation cancelled by user
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user


In [2]:
!pip install pytorch-lightning
  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
! cp /content/drive/MyDrive/DocRec/artifact/2_selected_sample_for_DCU_model.zip /content/

In [ ]:
! cp "/content/drive/MyDrive/DocRec/artifact/1000DataForOCR_fineLabel_dataset_coco_1.json" /content/

In [ ]:
! unzip /content/2_selected_sample_for_DCU_model.zip -d /content/

In [ ]:
from google.colab.patches import cv2_imshow


In [1]:
from transformers import LayoutXLMProcessor
import os

os.environ['CUDA_VISIBLE_DEVICES']='1'

processor = LayoutXLMProcessor.from_pretrained(
    "microsoft/layoutxlm-base", 
    apply_ocr=False, 
    only_label_first_subword=False, 
    is_split_into_words=True,
    )

/home/tiendq/.local/lib/python3.8/site-packages/transformers/models/layoutlmv2/feature_extraction_layoutlmv2.py:30: FutureWarning: The class LayoutLMv2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv2ImageProcessor instead.
  warnings.warn(


In [2]:
import os
import random
from enum import Enum

import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from pycocotools.coco import COCO
from PIL import Image, ImageDraw

color_map = {4: (0, 0, 0), 5: (255, 0, 0), 6: (0, 255, 0), 21: (0, 0, 255),
             22: (255, 255, 0), 23: (0, 255, 255), 24: (255, 0, 255), 25: (128, 128, 128),
             26: (192, 192, 192), 10: (64, 64, 64)}

label_list = ['title', 'explanation', 'answer', 'super_title', 'header', 'footer', 'ending', 'heading', 'starting']

def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]


def unnormalize_bbox(nbbox, width, height):
  return [
      int((nbbox[0] * width) / 1000),
      int((nbbox[1] * height) / 1000),
      int((nbbox[2] * width) / 1000),
      int((nbbox[3] * height) / 1000),
  ]


class CocoDataset(Dataset):
    def __init__(self, root_dir, annotation_file):
        self.root_dir = root_dir
        self.annotation_file = annotation_file

        self.coco = COCO(annotation_file)
        self.image_ids = list(sorted(self.coco.imgs.keys()))
        self.label_list = label_list
        self.label2CVATid = {self.coco.cats[_id]['name']: _id for _id in self.coco.cats.keys()}
        self.CVATid2label = {_id: self.coco.cats[_id]['name'] for _id in self.coco.cats.keys()}

        self.label2id = {label: idx for idx, label in enumerate(label_list)}
        self.id2label = {idx: label for idx, label in enumerate(label_list)}

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root_dir, img_info['file_name'])
        image = Image.open(img_path).convert('RGB')

        annotations_ids = self.coco.getAnnIds(imgIds=img_info['id'])
        annotations = self.coco.loadAnns(annotations_ids)
        # target['boxes'] = torch.Tensor([ann['bbox'] for ann in annotations])
        # target['labels'] = torch.LongTensor([ann['category_id'] for ann in annotations])

        bboxes = []
        labels_id = []
        words = []
        for ann in annotations:

            coco2normalbbox = [ann['bbox'][0], ann['bbox'][1], ann['bbox'][0] + ann['bbox'][2], ann['bbox'][1] + ann['bbox'][3]]

            # skip instance that is other than the label list in this problem
            try:
                box = normalize_bbox(coco2normalbbox, image.width, image.height)
                label_id = self.label2id[self.CVATid2label[ann['category_id']]]
                word = ann['attributes']['value']
            except KeyError:
                continue

            bboxes.append(box)

            labels_id.append(label_id)

            words.append(word)

        # return image, annotations
        # print(type(words), words)
        # encoding = processor(
        #     np.array(image), 
        #     words,
        #     boxes=torch.tensor(bboxes),
        #     word_labels=torch.tensor(labels_id),
        #     max_length=512,
        #     truncation=True,
        #     padding="max_length",
        #     # pad_to_multiple_of=8,
        #     return_tensors="pt"
        # )

        # return encoding
        # return dict(
        #     input_ids=encoding['input_ids'].flatten(),
        #     attention_mask=encoding['attention_mask'].flatten(),
        #     bbox=encoding['bbox'].flatten(end_dim=1),
        #     image=encoding['image'].flatten(end_dim=1),
        #     labels=encoding['labels'].flatten()
        # )

        # print(img_info)
        return {'words':words, 'boxes':bboxes, 'labels_id': labels_id, 'id': img_info['id'], 'width': img_info['width'], 'height': img_info['height'], 'image_path': self.root_dir + os.sep+ img_info['file_name']}

    def draw_example(self, words, boxes, labels_id, width, height, file_name, **kwargs):
        # Convert the image to a NumPy array and draw the annotations using cv2
        # self.root_dir
        image = Image.open(self.root_dir+os.sep+file_name).convert('RGB')
        image = np.array(image)
        for i in range(len(boxes)):
            bbox = boxes[i]
            bbox = unnormalize_bbox(bbox, width, height)

            cvatId = self.label2CVATid[self.id2label[labels_id[i]]]
            color = color_map.get(cvatId)

            cv2.rectangle(image, (int(bbox[0]), int(bbox[1])),
                          (int(bbox[2]), int(bbox[3])),
                          color, thickness=1)
            label = self.coco.loadCats(cvatId)[0]["name"]
            cv2.putText(image, label, (int(bbox[0]), int(bbox[1] - 2)), cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.25, color=(0, 0, 255), thickness=1)

        return image
        # cv2.imshow("Image with Annotations", image_np)
        # cv2.waitKey(0)


# if __name__ == "__main__":
root = "/home/tiendq/document-layout-analysis/0_data_repository/2_selected_sample"
ann_file = "/home/tiendq/document-layout-analysis/0_data_repository/1000DataForOCR_fineLabel_dataset_coco.json"
torch_dataset = CocoDataset(root, ann_file)
# dataset[0]
# print(len(dataset))
# cv2_imshow(dataset.draw_example(**dataset[57]))
# print(dataset.draw_example(2))


loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [ ]:
# import pandas as pd

# list_of_records = []
# for i in range(len(dataset)):
#     list_of_records.append(dataset[i])

# df = pd.DataFrame(list_of_records)
# # df.head()

In [ ]:
# train_size = int(0.8 * len(torch_dataset))
# test_size = len(torch_dataset) - train_size
# train_ds, test_ds = torch.utils.data.random_split(torch_dataset, [train_size, test_size])


In [3]:
from datasets import Dataset
# ds = Dataset.from_pandas(df)

def to_dataset():

    # for i in range(len(torch_dataset)):
    for i in range(100):

        yield torch_dataset[i]

ds = Dataset.from_generator(to_dataset)

Found cached dataset generator (/home/tiendq/.cache/huggingface/datasets/generator/default-39aee09cfd40e826/0.0.0)


In [4]:
ds = ds.train_test_split(test_size=0.2, shuffle=True)

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['words', 'boxes', 'labels_id', 'id', 'width', 'height', 'image_path'],
        num_rows: 80
    })
    test: Dataset({
        features: ['words', 'boxes', 'labels_id', 'id', 'width', 'height', 'image_path'],
        num_rows: 20
    })
})

In [5]:
features = ds["train"].features
column_names = ds["train"].column_names
image_column_name = "image"
text_column_name = "words"
boxes_column_name = "boxes"
label_column_name = "labels_id"


def prepare_examples(examples):
    images = [Image.open(path).convert("RGB") for path in examples['image_path']] 
    words = examples[text_column_name]
    boxes = examples[boxes_column_name]
    word_labels = examples[label_column_name]
    encoding = processor(images, words, boxes=boxes, word_labels=word_labels, truncation=True, stride =128, 
         padding="max_length", max_length=512, pad_token_label = -100, return_overflowing_tokens=True, return_offsets_mapping=True)
    offset_mapping = encoding.pop('offset_mapping')
    overflow_to_sample_mapping = encoding.pop('overflow_to_sample_mapping')
    return encoding

In [6]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'image': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})

train_dataset =  ds["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
eval_dataset = ds["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
     

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [8]:
eval_dataset

Dataset({
    features: ['image', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 5
})

In [9]:
train_dataset

Dataset({
    features: ['image', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 19
})

In [7]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")


In [8]:
from transformers import LayoutLMv2ForTokenClassification, LayoutXLMProcessor
import copy
import torch
import numpy as np


In [9]:
model = LayoutLMv2ForTokenClassification.from_pretrained('microsoft/layoutxlm-base', num_labels=len(torch_dataset.label_list), id2label=torch_dataset.id2label, label2id=torch_dataset.label2id)

# processor = LayoutXLMProcessor.from_pretrained("microsoft/layoutxlm-base", apply_ocr=False, only_label_first_subword=False, is_split_into_words=True)


Some weights of the model checkpoint at microsoft/layoutxlm-base were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.14.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.1.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.16.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.4.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.12.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.15.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.16.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.18.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.0.conv1.norm.num_batches_tracked', 'lay

In [13]:
import numpy as np
import torch
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        ["I-"+torch_dataset.label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        ["I-"+torch_dataset.label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)

    no_flatten = ['overall_precision', "overall_recall", 'overall_f1', 'overall_accuracy']
    to_be_flatten = []
    for key, val in results.items():
        if key in no_flatten:
            continue
        to_be_flatten.append(key)
    
    for key in to_be_flatten:
        print(key)
        val = results[key]
        for metr, value in val.items():
            results[key+"_"+metr] = value
        del results[key]

    return results
    # return {
    #     "precision": results["overall_precision"],
    #     "recall": results["overall_recall"],
    #     "f1": results["overall_f1"],
    #     "accuracy": results["overall_accuracy"],
    # }

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [14]:

from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import default_data_collator

training_args = TrainingArguments(
    output_dir='/home/tiendq/document-layout-analysis/0_model_repository',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    # warmup_steps=10,                # number of warmup steps for learning rate scheduler
    # weight_decay=0.01, 
    learning_rate=1e-5,
    evaluation_strategy="steps",
    eval_steps=5,              # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="overall_f1",
    resume_from_checkpoint=True
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,             # evaluation dataset
)

trainer.train()

Step,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Answer Precision,Answer Recall,Answer F1,Answer Number,Ending Precision,Ending Recall,Ending F1,Ending Number,Explanation Precision,Explanation Recall,Explanation F1,Explanation Number,Footer Precision,Footer Recall,Footer F1,Footer Number,Header Precision,Header Recall,Header F1,Header Number,Heading Precision,Heading Recall,Heading F1,Heading Number,Starting Precision,Starting Recall,Starting F1,Starting Number,Super Title Precision,Super Title Recall,Super Title F1,Super Title Number,Title Precision,Title Recall,Title F1,Title Number
5,1.618000,1.693942,0.000000,0.000000,0.000000,0.422444,0.000000,0.000000,0.000000,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,14,0.000000,0.000000,0.000000,35,0.000000,0.000000,0.000000,141
10,1.800800,1.557736,0.006596,0.051429,0.011692,0.455976,0.014019,0.082569,0.023968,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,14,0.004657,0.114286,0.008949,35,0.004072,0.035461,0.007305,141
15,1.438300,1.419508,0.006309,0.040000,0.010899,0.522898,0.008651,0.045872,0.014556,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,14,0.005076,0.085714,0.009585,35,0.005714,0.042553,0.010076,141
20,1.478900,1.382092,0.014650,0.088571,0.025142,0.592159,0.019174,0.119266,0.033037,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,14,0.000000,0.000000,0.000000,35,0.016949,0.127660,0.029925,141
25,1.143700,1.347125,0.040055,0.165714,0.064516,0.572168,0.037398,0.211009,0.063536,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,14,0.000000,0.000000,0.000000,35,0.048276,0.248227,0.080831,141
30,1.591200,1.251494,0.030922,0.151429,0.051357,0.625195,0.051173,0.220183,0.083045,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.000000,0.000000,0.000000,14,0.002825,0.028571,0.005141,35,0.033654,0.198582,0.057554,141
35,1.257700,1.229175,0.038936,0.171429,0.063458,0.534879,0.076923,0.284404,0.121094,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.021053,0.142857,0.036697,14,0.028818,0.285714,0.052356,35,0.024460,0.120567,0.040670,141
40,1.084800,1.200259,0.048684,0.211429,0.079144,0.520133,0.073222,0.321101,0.119250,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.032609,0.214286,0.056604,14,0.029508,0.257143,0.052941,35,0.042789,0.191489,0.069948,141
45,1.080700,1.080812,0.056849,0.237143,0.091713,0.669148,0.067720,0.275229,0.108696,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.030928,0.214286,0.054054,14,0.027650,0.171429,0.047619,35,0.062767,0.312057,0.104513,141
50,1.105200,1.163751,0.034556,0.140000,0.055430,0.540692,0.040323,0.137615,0.062370,109,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,18,0.000000,0.000000,0.000000,8,0.000000,0.000000,0.000000,3,0.000000,0.000000,0.000000,13,0.023256,0.214286,0.041958,14,0.042735,0.285714,0.074349,35,0.034146,0.148936,0.055556,141


KeyboardInterrupt: 

In [ ]:

import torch

example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

In [ ]:
for id, label in zip(train_dataset[0]["input_ids"], train_dataset[0]["labels"]):
  print(processor.tokenizer.decode([id]), label.item())

In [ ]:
# from torch.utils.data import DataLoader

# train_data_loader = DataLoader(
#     train_dataset,
#     batch_size=4,
#     shuffle=False,
#     num_workers=2
# )

# test_data_loader = DataLoader(
#     test_dataset,
#     batch_size=4,
#     shuffle=False,
#     num_workers=2
# )


In [ ]:
import pytorch_lightning as pl
from transformers import AdamW, LayoutLMv2ForTokenClassification

class ModelModule(pl.LightningModule):
    def __init__(self, dataset:CocoDataset):
        super().__init__()
        self.model = LayoutLMv2ForTokenClassification.from_pretrained('microsoft/layoutxlm-base', num_labels=len(dataset.label_list), id2label=dataset.id2label, label2id=dataset.label2id)
        self.dataset = dataset
    def forward(self, input_ids, attention_mask, bbox, image, labels=None):
        return self.model(input_ids, attention_mask=attention_mask, bbox=bbox, image=image, labels=labels)

    def training_step(self, batch, batch_idx):
        labels = batch['labels']
        outputs = self(
            batch['input_ids'],
            batch['attention_mask'],
            batch['bbox'],
            batch['image'],
            labels
            )
        loss = outputs.loss 
        self.log('train_loss', loss)
        self.log("train_overall_f1", compute_metrics(outputs.logits, labels, self.dataset)['f1'])
        return loss

    def validation_step(self, batch, batch_idx):
        labels = batch['labels']
        outputs = self(
            batch['input_ids'],
            batch['attention_mask'],
            batch['bbox'],
            batch['image'],
            labels
            )
        loss = outputs.loss 
        self.log('val_loss', loss)
        self.log("val_overall_f1", compute_metrics(outputs.logits, labels, self.dataset)['f1'])

        return loss

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=5e-5)
    

In [ ]:
model_module = ModelModule(dataset)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

early_stop_callback = EarlyStopping(monitor="val_accuracy", min_delta=0.00, patience=3, verbose=False, mode="max")
model_checkpoint = ModelCheckpoint(
    save_last=True,
    save_top_k=2,
    monitor="val_loss",
    mode='min'
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=16,
    devices=1,
    max_epochs=20,
    callbacks=[
        model_checkpoint,
        early_stop_callback
    ]
)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
trainer.fit(model_module, train_data_loader, test_data_loader)

In [ ]:
ds[0]['words']

In [ ]:
def tokenization(example):
    image = Image.open(dataset.root_dir + os.sep + example['file_name']).convert('RGB')
    # return processor(
    #         np.array(image), 
    #         example['words'],
    #         boxes=torch.tensor(example['boxes']),
    #         word_labels=torch.tensor(example['labels_id']),
    #         max_length=512,
    #         truncation=True,
    #         padding=True,
    #         # pad_to_multiple_of=8,
    #         return_tensors="pt"
    #     )
    example['image'] = np.array(image)
    return example

ds = ds.map(tokenization, batched=False)

In [ ]:
# implement a data colator fo

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()